# Weather Controller

This notebook publishes weather state changes to MQTT.
It cycles between sunny and rainy states:
- Sunny for 20 seconds
- Rain for 10 seconds
- Repeat every 30 seconds

**Usage:**
1. Run all cells to start the weather controller
2. The advanced map viewer and person walkers will react to weather changes

In [ ]:
import asyncio
import json
import time

from simulated_city.config import load_config
from simulated_city.mqtt import MqttConnector, MqttPublisher

In [ ]:
# MQTT setup
cfg = load_config()
connector = MqttConnector(cfg.mqtt, client_id_suffix="weather-controller")
connector.connect()
if not connector.wait_for_connection(timeout=10.0):
    raise RuntimeError("Failed to connect to MQTT broker")

publisher = MqttPublisher(connector)
print("✓ Connected to MQTT broker")

In [ ]:
# Weather cycle parameters
SUNNY_DURATION_S = 20.0
RAIN_DURATION_S = 10.0
CYCLE_PERIOD_S = SUNNY_DURATION_S + RAIN_DURATION_S  # 30 seconds total

async def weather_controller():
    """
    Publish weather state changes to MQTT.
    Cycles: sunny for 20s, rain for 10s, repeat.
    
    Publishes to: weather/status
    Message format: {"state": "sunny" | "rain", "timestamp": float}
    """
    print(f"\n🌤️  Weather cycle starting:")
    print(f"   Sunny: {SUNNY_DURATION_S}s")
    print(f"   Rain:  {RAIN_DURATION_S}s")
    print(f"   Cycle: {CYCLE_PERIOD_S}s\n")
    
    topic = "weather/status"
    cycle_count = 0
    
    while True:
        cycle_count += 1
        
        # Sunny phase
        message = {
            "state": "sunny",
            "timestamp": time.time(),
            "cycle": cycle_count
        }
        publisher.publish_json(topic, json.dumps(message), qos=0)
        print(f"[{time.strftime('%H:%M:%S')}] Cycle {cycle_count}: ☀️  SUNNY (20s)")
        await asyncio.sleep(SUNNY_DURATION_S)
        
        # Rainy phase
        message = {
            "state": "rain",
            "timestamp": time.time(),
            "cycle": cycle_count
        }
        publisher.publish_json(topic, json.dumps(message), qos=0)
        print(f"[{time.strftime('%H:%M:%S')}] Cycle {cycle_count}: 🌧️  RAIN (10s)")
        await asyncio.sleep(RAIN_DURATION_S)

In [ ]:
# Start the weather controller
task = asyncio.create_task(weather_controller())
print("✓ Weather controller is running!")
print("Run the next cell to stop.")

In [ ]:
# Stop the weather controller
task.cancel()
connector.disconnect()
print("✓ Weather controller stopped.")